In [ ]:
workdir_ = '/glade/work/juliob/'
import sys
#######################################
# Leave this for now. But it should change to better
# method as here:
# import os
# module_a_dir = os.path.dirname(os.path.abspath(__file__))
# utils_path = os.path.join(module_a_dir, '..', 'Utils')
# sys.path.append(utils_path)
# print( f" a path added in {__name__} {utils_path} ")
########################################
sys.path.append(workdir_ + 'myPythonTools/GenlTools/')
sys.path.append(workdir_ + 'myPythonTools/Utils/')
#sys.path.append(workdir_ + 'PyRegridding/Regridder/')
sys.path.append(workdir_ + 'PyRegridding/Utils/')

# Own local packages
import AveragingUtils as Av
#import VertRegridFlexLL as Vrg  # This is toxic for some reason
import PlotUtil as Pu
import validation_data as Val
import var_A_x_B as vAB
import MakePressures as MkP
import GridUtils as GrU
import utils as uti

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( uti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
importlib.reload(vAB)
importlib.reload(MkP)


In [ ]:
import matplotlib
print(matplotlib.__version__)
print(cartopy.__version__)

In [ ]:
import dask.system

# Get the number of physical CPU cores
physical_cores = dask.system.CPU_COUNT

print(f"Number of physical CPU cores: {physical_cores}")

# If you want logical cores (including hyper-threading), use:
import multiprocessing

logical_cores = multiprocessing.cpu_count()

print(f"Number of logical CPU cores: {logical_cores}")


In [ ]:
from dask.distributed import Client, LocalCluster

# Create a local cluster
cluster = LocalCluster()
client = Client(cluster)

# Check the number of workers and threads per worker
print(f"Number of workers: {len(client.scheduler_info()['workers'])}")
print(f"Threads per worker: {client.nthreads()}")


In [ ]:
print(f"Dashboard link: {client.dashboard_link}")

In [ ]:
%%time
##B = uti.MakeDict4Exp( exp='c64_005mm_ne30pg3_FMTHIST_CTL' ,

A = uti.MakeDict4Exp( exp='c64_005_ne30pg3_FMTHIST_CTL_x01' ,
                     user='juliob', subd='regridded' , 
                      hsPat='cam.h0a' , ymdPat='*' ,verbose=True, open_dataset=True )


In [ ]:

exp_Val='ERA5'

In [ ]:
########################
# Some massaging
########################

#ps_x =Dx.PS.values
lon_a=A.X.lon.values
lat_a=A.X.lat.values
lev_a=A.X.lev.values
zlev_a=-7.0*np.log( lev_a/1_000. )



In [ ]:

DstFileAlreadyExists = os.path.exists(DstFile)


In [ ]:
ueq = Dc.U[:,:,315:325,:].values

In [ ]:
####
# Validation
####


yearsA = '*'
ERA5dir = '/glade/campaign/cgd/amp/juliob/ERA5/Zonal_monthly/'
# get ERA5 pl 
path_C = ERA5dir + 'ERA5.native.time.' + yearsA +'-*.nc'
Dc = xr.open_mfdataset( path_C ,data_vars='all', coords='different') # , chunks=16 )
#print( f" Validation data {fld} has dims {Dc[fld].dims}")








#Uval = Val.data(fld='U',zlev=True, Timecube=True ) #, Years='2000' )
#UUv , zlev_v,lat_v,lon_v = Uval['aa'], Uval['lev'], Uval['lat'], Uval['lon']



In [ ]:
ug=Dc.U[:,:,320].values


In [ ]:
plt.contour(ug.T)

In [ ]:
import glob
import os

####
# Validation
####
yearsA = '*'
ERA5dir = '/glade/campaign/cgd/amp/juliob/ERA5/native_monthly/'
# get ERA5 pl 
path_C = ERA5dir + 'ERA5.native.time.' + yearsA +'-*.nc'
DstDir = ERA5dir.replace('/native_monthly/', '/Zonal_monthly/' )

#######
os.makedirs( DstDir , exist_ok=True )

# Get the list of files matching the pattern
files = glob.glob(path_C)

# Sort the files alphabetically
sorted_files = sorted(files)

# Print the sorted list of files
print("Sorted files:")
for file in sorted_files:
    filZ = file.replace('/native_monthly/', '/Zonal_monthly/' )
    X = xr.open_dataset( file ) #, chunks=16 )
    Xz = X.mean(dim='longitude')
    Xz.to_netcdf( filZ )
    print(f'Wrote {filZ}')

#Dc = xr.open_mfdataset( path_C ,data_vars='all', coords='different' , chunks=16 )


In [ ]:
Zc = Dc.mean(dim='longitude')




In [ ]:
Zc.to_netcdf( '/glade/derecho/scratch/juliob/' + 'ERA5.native.time.ZonalMean.nc' )

In [ ]:
plt.plot( ZU[:,50,320] )

In [ ]:
#ZU


# Assuming 'data' is your DataArray with a 'lat' dimension
# Subset the data between 5S and 5N
EU = ZU.sel(latitude=slice(-5, 5))

# Calculate the mean over the latitude dimension
QBOU = EU.mean(dim='latitude')


In [ ]:
qbou = QBOU.values

In [ ]:
plt.plot( qbou.flatten() )


In [ ]:
fig = plt.figure(figsize=(24, 5))
clev=np.linspace( -60,140,num=21) 
dlev=np.linspace( -20,20,num=21) 
cmap='gist_ncar'


Axes1 = Pu.axes_def(n=1,nxplo=4,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a,UUa_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a,UUa_sz ,levels=clev, colors='black')
ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)


Axes1 = Pu.axes_def(n=3,nxplo=4,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_c,zlev_c,UUc_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat_c,zlev_c,UUc_sz ,levels=clev, colors='black')
ax1.set_title(f"Control <{C.exp}> {season.upper()} {years_c[0]}-{years_c[-1]}" )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)
"""
Axes1 = Pu.axes_def(n=3,nxplo=4,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_c,zlev_c,UUd_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat_c,zlev_c,UUd_sz ,levels=clev, colors='black')
ax1.set_title(f"Control <{D.exp}> {season.upper()} {years_d[0]}-{years_d[-1]}" )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)
"""

Axes1 = Pu.axes_def(n=2,nxplo=4,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_b,zlev_b,UUb_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat_b,zlev_b,UUb_sz ,levels=clev, colors='black')
ax1.set_title(f"Test <{exp_B}> {season.upper()} {years_b[0]}-{years_b[-1]}" )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)


Axes1 = Pu.axes_def(n=4,nxplo=4,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_v,zlev_v, UUv_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat_v,zlev_v, UUv_sz ,levels=clev, colors='black')
ax1.set_title(f"Validation <{exp_Val}> {season.upper()}  " )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)


In [ ]:
#Below here we are looking at GW mom flux


In [ ]:
####

In [ ]:
#season='jul'
OUc_s,years_c,months_c = Av.Seasonal( ds=C.X, season=season , fld='OMEGAU',return_time=True)
OUb_s,years_b,months_b = Av.Seasonal( ds=B.X, season=season , fld='OMEGAU',return_time=True)
OUa_s,years_a,months_a = Av.Seasonal( ds=A.X, season=season , fld='OMEGAU',return_time=True)

Oc_s,years_c,months_c = Av.Seasonal( ds=C.X, season=season , fld='OMEGA',return_time=True)
Ob_s,years_b,months_b = Av.Seasonal( ds=B.X, season=season , fld='OMEGA',return_time=True)
Oa_s,years_a,months_a = Av.Seasonal( ds=A.X, season=season , fld='OMEGA',return_time=True)

In [ ]:
OUpa_s = OUa_s - Oa_s*UUa_s
OUpb_s = OUb_s - Ob_s*UUb_s
OUpc_s = OUc_s - Oc_s*UUc_s


OUpa_sz=np.average( OUpa_s , axis=2 )
OUpb_sz=np.average( OUpb_s , axis=2 )
OUpc_sz=np.average( OUpc_s , axis=2 )



In [ ]:
fig = plt.figure(figsize=(20, 5))
clev=0.5*np.linspace( -0.1,0.1,num=21) 
dlev=21 #np.linspace( -20,20,num=21) 
cmap='gist_ncar'
exp_A=A.exp
fldoo="$\overline{u'\omega'}$"

n=1
Axes1 = Pu.axes_def(n=n,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a,OUpa_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a,OUpa_sz ,levels=clev, colors='black')
ax1.set_title(f"{fldoo} <{exp_A}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_b,zlev_b,OUpb_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat_b,zlev_b,OUpb_sz ,levels=clev, colors='black')
ax1.set_title(f"{fldoo} <{exp_B}> {season.upper()} {years_b[0]}-{years_b[-1]}" )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_c,zlev_c,OUpc_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat_c,zlev_c,OUpc_sz ,levels=clev, colors='black')
ax1.set_title(f"Control <{exp_C}> {season.upper()} {years_c[0]}-{years_c[-1]}" )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)



"""
Axes1 = Pu.axes_def(n=4,nxplo=4,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_v,zlev_v, UUv_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat_v,zlev_v, UUv_sz ,levels=clev, colors='black')
ax1.set_title(f"Validation <{exp_Val}> {season.upper()}  " )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)
"""

In [ ]:
C.X